In [4]:
import datetime as dt
import json
import pandas as pd # type: ignore
import numpy as np # type: ignore
import re
import os
import time
import sys
import logging
from google.cloud import bigquery
import warnings


sys.path.insert(0,os.environ["HOME"]+"/trading/python/")
from lib import aplo # type: ignore
from lib import talos_utils # type: ignore
from lib import vault_utils # type: ignore

sys.path.append(os.environ["HOME"] + "/anchorage/source/python/lib/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility # type: ignore
import utils as aql_utils # type: ignore

sys.path.append(os.environ["HOME"]+"/trading/keys/")
import keys

logging.getLogger().setLevel(logging.INFO)
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.8f}'.format
warnings.filterwarnings("ignore", category=UserWarning)

In [5]:
sys.path.append(os.path.expanduser("~") + "/anchorage/source/python/trading/agency_desk")
from lib import talos_utils

sys.path.append(os.environ["HOME"] + "/anchorage/source/python/lib/quant_lib/anchoragequantlib")
import google_sheet_utility as aql_google_sheet_utility # type: ignore
import utils as aql_utils # type: ignore


pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.8f}'.format

api_key = 'ANC34AV1RAK2'
api_secret = 'mj9a70zpeve9rnam55kn0ql6c7sze3gw'

talos_wl_api = 'ANC34AV1RAK2'
talos_wl_secret_api = 'mj9a70zpeve9rnam55kn0ql6c7sze3gw'
utc_now = datetime.datetime.utcnow()
utc_datetime = utc_now.strftime("%Y-%m-%dT%H:%M:%S.000000Z")

talos_api = 'ANCHJEWQR3YH'
talos_secret = '3c4nnc7lg7mfg2ae2irz5kkueeei37pz'

host = "tal-42.prod.talostrading.com"
host_wl = "tal-160.prod.talostrading.com"

talos = talos_utils.Talos(talos_api, talos_secret, host)
talos_wl = talos_utils.Talos(talos_wl_api, talos_wl_secret_api, host_wl)



In [11]:
os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
google_sheet_key = aql_utils.read_secret(os.environ.get("GOOGLE_SHEET_KEY"))
gsheet_key = json.loads(google_sheet_key)
worksheet_name = "MSTR Trading Dashboard"
tab_name = "stream"
gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
dealers = ['cumberland']

In [25]:
def get_size_prices_spread(symbol, exchanges=[], dealers=['cumberland'], liquidity_type="Firm", depth: str = "VWAP", **kwargs):

    try:
        # Assuming talos.get_market_data_feed_v2() retrieves market data
        talos_stream = talos.get_market_data_feed_v2(symbol=symbol, exchanges=exchanges, dealers=dealers, LiquidityType=liquidity_type, **kwargs)
        #filtered_data = {key: value for key, value in talos_stream.items() if any(d['DepthType'] for d in value)}
        filtered_data = {key:value for key, value in talos_stream.items() if sorted(value[0]['Markets'].keys())==sorted(exchanges)}

        first_available_key = next(iter(filtered_data))
        talos_stream = filtered_data[first_available_key]
        
        # Extract bid and offer prices
        bids = talos_stream[0]["Bids"]
        offers = talos_stream[0]["Offers"]

        # Create DataFrame for bid and offer prices
        bid_df = pd.DataFrame(bids).T
        offer_df = pd.DataFrame(offers).T

        # Concatenate bid and offer DataFrames
        size_prices = pd.concat([bid_df, offer_df], axis=1)
        
        # Check if size_prices has only one column
        if len(size_prices.columns) == 1:
            # Handle the situation where there's only one column
            size_prices.columns = ["Price"]
        else:
            # Ensure that the number of columns matches the expected number of elements in the column names list
            if len(size_prices.columns) != len(["Bid", "Offer"]):
                # If the number of columns doesn't match, assign generic column names
                size_prices.columns = [f"Column_{i}" for i in range(len(size_prices.columns))]
            else:
                # If the number of columns matches, assign the provided column names
                size_prices.columns = ["Bid", "Offer"]

        # Convert prices to float
        size_prices[["Bid", "Offer"]] = size_prices[["Bid", "Offer"]].astype(float)

        # Calculate mid price and spread
        size_prices["Mid"] = (size_prices["Bid"] + size_prices["Offer"]) / 2
        size_prices["Spread"] = ((size_prices["Offer"] - size_prices["Bid"]) / size_prices["Bid"]) * 10000
        size_prices["BidSpread"] = ((size_prices["Mid"] - size_prices["Bid"]) / size_prices["Mid"]) * 10000
        size_prices["AskSpread"] = ((size_prices["Offer"] - size_prices["Mid"]) / size_prices["Mid"]) * 10000

        # Select bid, mid, and offer prices for the specified depth
        depth_prices = size_prices.loc[depth][["Bid", "Mid", "Offer","Spread","BidSpread","AskSpread"]]

        # Create DataFrame with coin, bid_price, mid price, and offer_price
        result_df = pd.DataFrame({
            'coin': [symbol],
            'bid_price': [depth_prices['Bid']],
            'mid_price': [depth_prices['Mid']],
            'offer_price': [depth_prices['Offer']],
            'spread_bps': [depth_prices['Spread']],
            'bid_bps': [depth_prices['BidSpread']],
            'ask_bps': [depth_prices['AskSpread']]
        })

        return result_df
    except KeyError:
        print(f"Error for asset {symbol}")
        return pd.DataFrame({'coin': [symbol], 'bid_price': [np.nan], 'mid_price': [np.nan], 'offer_price': [np.nan], 'spread_bps': [np.nan], 'bid_bps': [np.nan], 'ask_bps': [np.nan]})


quote_results = []

symbol = "BTC-USD"
#size_buckets = [row['usd_notional']]  # Ensure size_buckets is a list

    # Call generate_quotes function with symbol and delta
quote_result = get_size_prices_spread(symbol, AmountBuckets="1", liquidity_type="Firm")
quote_results.append(quote_result)

display(quote_results)
df = pd.DataFrame(quote_results)


os.environ["GOOGLE_SHEET_KEY"] = "projects/375663101687/secrets/trading_gsheet_auth_token/versions/1"
google_sheet_key = aql_utils.read_secret(os.environ.get("GOOGLE_SHEET_KEY"))
gsheet_key = json.loads(google_sheet_key)
worksheet_name = "MSTR Trading Dashboard"
tab_name = "stream"

gsu = aql_google_sheet_utility.GoogleSheetUtility(gsheet_key, worksheet_name)
gsu.dump_current_sheet(tab_name, quote_results)



[2025-01-22T02:30:45.341+0000] {talos_utils.py:260} INFO - Fetching for BTC-USD on [] and ['cumberland'] | {'LiquidityType': 'Firm', 'AmountBuckets': '1'}
Error for asset BTC-USD


[      coin  bid_price  mid_price  offer_price  spread_bps  bid_bps  ask_bps
 0  BTC-USD        NaN        NaN          NaN         NaN      NaN      NaN]

ValueError: Must pass 2-d input. shape=(1, 1, 7)

['04c74be4-6497-4439-bcb8-9b98ffd6e3b6', '7c5f2da4-7a4c-4ef8-a842-d3d00ddec8d3', 'c96c5c12-decc-49b9-b86d-e3f9212b3888', '5e4a3efa-ff04-415c-890b-e92a44d56cf2', '5ae00a3d-9c0c-44ce-a3b6-013ee86d3c55', 'ad35ed7f-7f24-41b3-92b1-39a6d8533dbc', '437d524b-fbfc-40e4-a074-949e28f462e5', 'a059cc2d-a436-48cc-88a3-62f17b2eb708', '0eb64ba3-f34b-4065-8807-2948f833e9f9', '2fe6d6bb-a004-49a8-93f2-7cb0c9643351', '9b0950b1-8eb4-40d7-b4c6-dc5aa4ee5d15', '6374c204-94a2-48c8-a2f0-5ceb8160f06c', '3ce7ef52-536d-4711-a628-dabefb51795f', '55363921-2a52-4f1a-a4e4-46cc162f53bb', 'c9d4991b-282e-4c62-a609-6a6e4810bd38', '9da85b2e-4cf8-47be-a176-ba1f1118de8b', '5992f81b-e046-4dff-a1f3-5d254e12445c', 'f03193b8-d6c6-43cc-8655-ecfa3e2f0bda', 'ac4ac969-be8b-4543-972d-43303b552eea', '8ad0b759-8461-4597-b46d-4aef3109b610', 'aa35f820-0b5d-4457-a86f-3e6d5f980199', '8de3e9ba-33d9-4bef-b84e-b73406bfea89']


AttributeError: 'DataFrame' object has no attribute 'Timestamp'